In [ ]:
# Application_train table path
APPLICATION_TRAIN_DATA_PATH = '../data/raw/application_train.csv'

# Bureau table path 
BUREAU_DATA_PATH = '../data/raw/bureau.csv'

# Previous_application table path 
PREVIOUS_APPLICATION_DATA_PATH = '../data/raw/previous_application.csv'

# Interim data directory
INTERIM_DIR = '../data/interim'

# Processed data directory
PROCESSED_DIR = '../data/processed'

# Model Directory
MODEL_DIR = '../models'

In [ ]:
df = pd.read_csv(APPLICATION_TRAIN_DATA_PATH)

df_bureau = pd.read_csv(BUREAU_DATA_PATH)

df_previous_application = pd.read_csv(PREVIOUS_APPLICATION_DATA_PATH)

## 1. Preprocess Data 1

In [ ]:
# remove the four rows with 'XNA' gender due to insufficient data and imbalance
df = df[(df['CODE_GENDER'] == 'F') | (df["CODE_GENDER"] == 'M')]

In [ ]:
# Select candidate columns that contain meaningful data and drop others
cleaned_df = df.loc[:, ['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER',
                    'FLAG_OWN_CAR','FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
                    'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'NAME_TYPE_SUITE',
                    'NAME_INCOME_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE',
                    'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
                    'OWN_CAR_AGE','OCCUPATION_TYPE', 'CNT_FAM_MEMBERS', 'ORGANIZATION_TYPE',
                    'EXT_SOURCE_2', 'EXT_SOURCE_3', 'LIVINGAREA_AVG','TOTALAREA_MODE',
                    'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE',
                    'DEF_60_CNT_SOCIAL_CIRCLE', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR',
                    'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH','FLAG_MOBIL',
                    'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE','FLAG_EMAIL',
                    'REGION_RATING_CLIENT','REGION_RATING_CLIENT_W_CITY',
                    'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION',
                    'LIVE_REGION_NOT_WORK_REGION','DAYS_LAST_PHONE_CHANGE'
                  ]
              ]

In [ ]:
# split the dataframe into numerical data and categorical data
cat = cleaned_df.select_dtypes('object')
num = cleaned_df.select_dtypes(include=['int64','float64'])

In [ ]:
# Rename specific columns related to number of days
num = num.rename(columns={"DAYS_BIRTH": "YEARS_BIRTH",'DAYS_EMPLOYED':'YEARS_EMPLOYED','DAYS_REGISTRATION':'YEARS_REGISTRATION','DAYS_ID_PUBLISH':'YEARS_ID_PUBLISH','DAYS_LAST_PHONE_CHANGE':'YEARS_LAST_PHONE_CHANGE'})

# Calculate the number of years for each column in days
num['YEARS_BIRTH'] = num['YEARS_BIRTH'].abs()/365
num['YEARS_EMPLOYED'] = num['YEARS_EMPLOYED'].abs()/365
num['YEARS_REGISTRATION'] = num['YEARS_REGISTRATION'].abs()/365
num['YEARS_ID_PUBLISH'] = num['YEARS_ID_PUBLISH'].abs()/365
num['YEARS_LAST_PHONE_CHANGE'] = num['YEARS_LAST_PHONE_CHANGE'].abs()/365

In [ ]:
# Change the noise values to the median, as the current values are nonsensical.
num["YEARS_EMPLOYED"][num["YEARS_EMPLOYED"]>1000] = num["YEARS_EMPLOYED"].median()

In [ ]:
# Dropping unnecessary columns
num.drop(['AMT_REQ_CREDIT_BUREAU_QRT','FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_MOBIL', 'FLAG_CONT_MOBILE', 'FLAG_EMAIL', 'YEARS_LAST_PHONE_CHANGE'], axis=1, inplace=True)

In [ ]:
# Fill five missing values of 'OWN_CAR_AGE' column with the mode value
num[(num['OWN_CAR_AGE'].isna()) & (cat["FLAG_OWN_CAR"] == 'Y')] = num[(num['OWN_CAR_AGE'].isna()) & (cat["FLAG_OWN_CAR"] == 'Y')].fillna(7)

In [ ]:
# fill nulls with median for specific columns based on observations and null percentages
'''
    ['DEF_60_CNT_SOCIAL_CIRCLE','DEF_30_CNT_SOCIAL_CIRCLE','OBS_60_CNT_SOCIAL_CIRCLE',
    'OBS_30_CNT_SOCIAL_CIRCLE', 'EXT_SOURCE_2', 'AMT_GOODS_PRICE','AMT_ANNUITY']
'''

num[['DEF_60_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE', 'OBS_30_CNT_SOCIAL_CIRCLE', 'EXT_SOURCE_2', 'AMT_GOODS_PRICE', 'AMT_ANNUITY', 'AMT_REQ_CREDIT_BUREAU_YEAR']] = num[['DEF_60_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE', 'OBS_30_CNT_SOCIAL_CIRCLE', 'EXT_SOURCE_2', 'AMT_GOODS_PRICE', 'AMT_ANNUITY', 'AMT_REQ_CREDIT_BUREAU_YEAR']].fillna(num[['DEF_60_CNT_SOCIAL_CIRCLE','DEF_30_CNT_SOCIAL_CIRCLE','OBS_60_CNT_SOCIAL_CIRCLE','OBS_30_CNT_SOCIAL_CIRCLE', 'EXT_SOURCE_2', 'AMT_GOODS_PRICE','AMT_ANNUITY','AMT_REQ_CREDIT_BUREAU_YEAR']].median())

In [ ]:
mf = MissForest(rgr=RandomForestRegressor())
num_imputed = mf.fit_transform(num[['LIVINGAREA_AVG', 'TOTALAREA_MODE', 'EXT_SOURCE_3']])

In [ ]:
# Update num dataframe with the imputed values using the RandomForestAlgorithm
num[['LIVINGAREA_AVG']] = num_imputed[['LIVINGAREA_AVG']]
num[['TOTALAREA_MODE']] = num_imputed[['TOTALAREA_MODE']]
num[['EXT_SOURCE_3']] = num_imputed[['EXT_SOURCE_3']]

In [ ]:
df_concat = pd.concat([num, cat], axis=1)

## 2. Preprocess Data 2

In [ ]:
df_application_train = df_concat

In [ ]:
df_application_train['OCCUPATION_TYPE'].fillna(value='XNA', inplace=True)

In [ ]:
df_application_train['OWN_CAR_AGE'].fillna(value=0, inplace=True)

In [ ]:
''' 
Create a new feature by combining two existing features, 
calculating an overall rating for a specific region that includes the city. 
'''

df_application_train['OVERALL_REGION_RATING'] = df_application_train['REGION_RATING_CLIENT_W_CITY']/df_application_train['REGION_POPULATION_RELATIVE']

In [ ]:
df_application_train['LOAN_REPAYMENT_PERIOD'] = df_application_train['AMT_CREDIT'] / df_application_train['AMT_ANNUITY']

In [ ]:
df_application_train = df_application_train.drop(['CNT_FAM_MEMBERS', 'AMT_CREDIT',
                                                'LIVINGAREA_AVG', 'OBS_30_CNT_SOCIAL_CIRCLE',
                                                'DEF_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE',
                                                'REGION_RATING_CLIENT','LIVE_REGION_NOT_WORK_REGION','NAME_TYPE_SUITE',
                                                'REG_REGION_NOT_LIVE_REGION', 'YEARS_REGISTRATION', 'YEARS_ID_PUBLISH', 'REG_REGION_NOT_WORK_REGION'], axis=1)

### 3.4 Handle Outliers

In [ ]:
# Drop the row having an anomaly 'AMT_INCOME_TOTAL' 
df_application_train.drop(df_application_train[df_application_train.AMT_INCOME_TOTAL > 100000000].index, inplace=True)

### 3.5 Aggregate Secondary Tables

#### 3.6.1 Bureau Table

In [ ]:
total_loans = df_bureau.groupby('SK_ID_CURR')['SK_ID_BUREAU'].count().reset_index()
total_loans.rename(columns={'SK_ID_BUREAU': 'TOTAL_LOANS'}, inplace=True)

In [ ]:
average_delay = df_bureau.groupby('SK_ID_CURR')['CREDIT_DAY_OVERDUE'].mean().reset_index()
average_delay.rename(columns={'CREDIT_DAY_OVERDUE': 'AVG_REPAYMENT_DELAY'}, inplace=True)

#### 3.6.2 Previous_Application Table

In [ ]:
count_previous_loans = df_previous_application.groupby('SK_ID_CURR')['SK_ID_PREV'].count().reset_index()
count_previous_loans.rename(columns={'SK_ID_PREV': 'COUNT_PREVIOUS_LOANS'}, inplace=True)

In [ ]:
average_loan_amount = df_previous_application.groupby('SK_ID_CURR')['AMT_APPLICATION'].mean().reset_index()
average_loan_amount.rename(columns={'AMT_APPLICATION': 'AVG_LOAN_AMOUNT'}, inplace=True)

#### 3.6.3 Merging Tables

In [ ]:
# Merging total loans
df_application_train = df_application_train.merge(total_loans, on='SK_ID_CURR', how='left')

# Merging average repayment delay
df_application_train = df_application_train.merge(average_delay, on='SK_ID_CURR', how='left')

In [ ]:
# Merging count of previous loans
df_application_train = df_application_train.merge(count_previous_loans, on='SK_ID_CURR', how='left')

# Merging average loan amount
df_application_train = df_application_train.merge(average_loan_amount, on='SK_ID_CURR', how='left')

#### 3.6.4 Cleaning After Merging

In [ ]:
# Filling missing values (if no loans exist for a customer)
df_application_train['TOTAL_LOANS'].fillna(0, inplace=True)
df_application_train['AVG_REPAYMENT_DELAY'].fillna(0, inplace=True)

In [ ]:
# Filling missing values (if no previous loans exist for a customer)
df_application_train['COUNT_PREVIOUS_LOANS'].fillna(0, inplace=True)
df_application_train['AVG_LOAN_AMOUNT'].fillna(0, inplace=True)

### 3.6 Normalize Data

    Utilize a robust scaler since the data contains significant outliers that should not be clipped.

In [ ]:
numerical = df_application_train.select_dtypes(['int64','float64'])
categorical = df_application_train.select_dtypes('object')

In [ ]:
numerical_columns = list(numerical.columns)
categorical_columns = list(categorical.columns)

In [ ]:
scaler = RobustScaler()
scaled_data = scaler.fit_transform(numerical)

In [ ]:
df_scaled = pd.DataFrame(scaled_data, columns= numerical_columns)

### 3.7 Prepare Data

In [ ]:
df_scaled.drop(['SK_ID_CURR', 'REGION_RATING_CLIENT_W_CITY'], axis=1, inplace=True)

In [ ]:
df_processed = pd.concat([df_scaled, categorical], axis=1)